Classification of clothing items for the task of [iMaterialist (Fashion) 2019 at FGVC6](http://https://www.kaggle.com/c/imaterialist-fashion-2019-FGVC6) dataset on Keras.


**Loading Libraries**

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2
plt.style.use("ggplot")

from tqdm import tqdm_notebook, tnrange, tqdm
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

from PIL import Image
import pandas as pd
import gc
import glob
import shutil
import json
from imgaug import augmenters as iaa
import imgaug as ia
from pathlib import Path
import seaborn as sns

In [ ]:
import tensorflow as tf
from keras import backend as K
import keras

from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.losses import binary_crossentropy
from keras.engine import InputSpec
from keras.engine.training import Model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import Sequence

%matplotlib inline

**Defining some important variables**

In [ ]:
img_height = 512 
img_width = 512 
img_channels = 3 
batch_size = 8

In [ ]:
seed = 10
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [ ]:
input_dir = "../input/imaterialist-fashion-2019-FGVC6/"
image_dir = "../input/imaterialist-fashion-2019-FGVC6/train/"
model_weights = "../input/weights-unet/unet_seresnet50_weights.h5"

**Data analysis**

* There are 333,415 total data.
* Considering only one unique image, the number of images is 45,625 total.
* One image can correspond to multiple classes.
* Not all clothing items are marked at the photos.
* There are 46 categories and 92 attributes.

In [ ]:
train_df = pd.read_csv(input_dir + "train.csv")
print("Number of data items: ", train_df.shape[0])
print("Number of unique images:",len(set(train_df['ImageId'])))

json_data = open(input_dir + "label_descriptions.json").read()
label_descriptions = json.loads(json_data)
categories_label_df = pd.DataFrame(label_descriptions['categories'])
print("The number of categories: ",len(categories_label_df))
classes_names = [x['name'] for x in label_descriptions['categories']]
print(classes_names)

attributes_label_df = pd.DataFrame(label_descriptions['attributes'])
print("The number of attributes: ",len(attributes_label_df))

In [ ]:
train_df_ImageId_count = train_df['ImageId'].value_counts()
plt.figure(figsize=(20, 7))
plt.title('image labels count', size=20)
plt.xlabel('', size=15);plt.ylabel('', size=15);
sns.countplot(train_df_ImageId_count)
plt.show()

In [ ]:
train_df['Category'] = train_df['ClassId'].apply(lambda x: int(x.split("_")[0]))
groupby_category = train_df.groupby('Category')['ImageId'].count()
groupby_category.index = map(int, groupby_category.index)
groupby_category = groupby_category.sort_index()
groupby_category[:5]

fig = plt.figure(figsize=(20, 7))
x = groupby_category.index
y = groupby_category.values

sns.barplot(x,y)
plt.title("Number of data items by category", fontsize=20)
plt.xlabel("Category", fontsize=20)
plt.ylabel("# of masks", fontsize=20)
plt.show()

In [ ]:
label_description_categories = pd.DataFrame(label_descriptions['categories'])

label_merge = categories_label_df[['id', 'name']].astype(str).astype(object)
train_classid = pd.DataFrame({'ClassId':train_df['ClassId'].apply(lambda x: x[:2].replace('_', ''))})
label_merge = label_description_categories[['id', 'name']].astype(str).astype(object)


train_df_name = train_classid.merge(label_merge, left_on='ClassId', right_on='id', how='left')
sum1 = train_df_name.shape[0]
ratio1 = np.round(train_df_name.groupby(['ClassId', 'name']).count().sort_values(by='id', ascending=False).rename(columns = {'id':'count'})/sum1 * 100, 2)
train_df_name_stat = train_df_name.groupby(['ClassId', 'name']).count().sort_values(by='id', ascending=False).rename(columns = {'id':'count'}).reset_index()
train_df_name_stat['ratio(%)'] = ratio1.values
train_df_name_stat = train_df_name_stat.head(22)

classes = [x for x in train_df_name_stat['ClassId']]
print(classes)
n_classes = len(classes)

In [ ]:
int_to_classes = {k:v for k, v in zip(range(0, len(classes)), classes)}
classes_to_int = {v:k for k, v in zip(range(0, len(classes)), classes)}

In [ ]:
df = pd.read_csv(input_dir + "train.csv")
df = df[df['ClassId'].isin(classes)].head(20000)
df['CategoryId'] = df.ClassId.apply(lambda x: str(x).split("_")[0])
temp_df = df.groupby('ImageId')['EncodedPixels', 'CategoryId'].agg(lambda x: list(x)).reset_index()
size_df = df.groupby('ImageId')['Height', 'Width'].mean().reset_index()
df = temp_df.merge(size_df, on='ImageId', how='left')
df.head()

**Loading data**

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes)

In [ ]:

class DataLoader():
    
    def __init__(self, img_dir, df, visualize=True):
        
        self.img_dir = img_dir
        self.df = df
        self.images = []
        self.labels = []
        self.visualize = visualize
    
    def get_data(self):       

        for index, row in tqdm(self.df.iterrows(), total=len(self.df)):
            
            image_id = row['ImageId']
            image_path = os.path.join(self.img_dir, image_id)
            self.images.append(image_path)
            
            mask = []
            img_labels = []
            for m, (annotation, label) in enumerate(zip(row['EncodedPixels'], row['CategoryId'])):
                img_labels.append(classes[classes_to_int[label]])

            img_labels = mlb.fit_transform([convert(img_labels)])
            self.labels.append(img_labels)
            
            if self.visualize:
                if index % 100 == 0:
                    self.visualize_results(image_path, img_labels)
                    
        return self.images, self.labels
                    
    
    def rle_decode(self, mask_rle, shape):
        shape = (shape[1], shape[0])
        s = mask_rle.split()
        starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
        starts -= 1
        ends = starts + lengths
        img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
        for lo, hi in zip(starts, ends):
            img[lo:hi] = 1
        return img.reshape(shape).T 
    
    
    def visualize_results(self, image_path, mask):
        img = Image.open(image_path).convert("RGB")
        plt.imshow(img)
        print(mask)
        mask = [mlb.classes_[i] for i in mask]
        plt.title(mask)
        plt.show()

    def get_datalen(self):
        return len(self.images)

In [ ]:
def convert(list): 
    return (*list, ) 

dataloader = DataLoader(image_dir, df, False)
images, labels = dataloader.get_data()
print(len(images))
print(len(labels))
print(mlb.classes)
print(labels[0])

**Splitting data into train and test**

In [ ]:
train_image, valid_image, train_labels, valid_labels = train_test_split(
        images, labels,
        test_size = 0.3, 
        random_state=42
)

print('{} training images && {} labels'.format(len(train_image), len(train_labels)))
print('{} validation images && {} labels'.format(len(valid_image), len(valid_labels)))

**Data Generator**

In [ ]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size
        
    def __len__(self):
        return len(self.image_filenames) // self.batch_size
    
    def __getitem__(self, idx):
        
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        batch_X = []
        batch_Y = []

        for filename in batch_x:
            with Image.open(filename).convert('RGB') as img:
                img = cv2.resize(np.asarray(img), (img_height, img_width))
                img = np.asarray(img) / 255.0
                batch_X.append(img)

        batch_X = np.asarray(batch_X)
        batch_Y = np.asarray(np.squeeze(batch_y, axis=1))

        return batch_X, batch_Y

In [ ]:
train_generator = DataGenerator(train_image, train_labels, batch_size)
valid_generator = DataGenerator(valid_image, valid_labels, batch_size)

In [ ]:
img, labels = train_generator.__getitem__(1)
print(labels.shape)
print(img.shape)

**Building a model**


In [ ]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes, finalAct="softmax"):
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
 
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
        
        # CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding="same",
            input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.25))

        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
 
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Dense(classes))
        model.add(Activation(finalAct))

        # return the constructed network architecture
        return model

In [ ]:
model = SmallerVGGNet.build(
    width=img_width, height=img_height,
    depth=img_channels, classes=len(mlb.classes_),
    finalAct="sigmoid")

opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0)

model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
class CustomCallback(Callback):
    def __init__(self, model):
        self.batch_size = 10
        self.generator = DataGenerator(valid_image, valid_labels, self.batch_size)
        self.model = model

    def on_epoch_end(self, epoch, logs={}):
        
        X, y_true = self.generator.__getitem__(0)
        threshold = 0.5
        
        fig, ax = plt.subplots(10, 2, figsize = (40, 60))
        
        y_pred = model.predict(X)
        count = 0
        
        for i in range(10):
            img = X[i]
            label_true = y_true[i]
            label_pred = y_pred[i]

            ax[i][count].imshow(img)
            true_idxs = np.where(label_true == 1)[0]
            true_label = [classes_names[int(classes[j])] for j in true_idxs]
            ax[i][count].set_title(true_label)
            ax[i][count].axis('off')
            
            idxs = np.argsort(label_pred)[::-1][:5]
            labels = []

            for (k, j) in enumerate(idxs):
                label = "{}: {:.2f}%".format(classes_names[int(classes[j])], label_pred[j] * 100)
                labels.append(label)

            ax[i][count + 1].imshow(img)
            ax[i][count + 1].set_title(labels)
            ax[i][count + 1].axis('off')
        
        plt.show()

**Training the model**

In [ ]:
epochs = 25
keras.backend.get_session().run(tf.global_variables_initializer())

history = model.fit_generator(
      generator = train_generator,
      epochs = epochs,
      callbacks = [
           CustomCallback(model),
           ModelCheckpoint('unet_weights_epoch-{epoch:02d}_loss-{loss:.4f}.h5',
                           monitor='val_loss',
                           verbose=1,
                           save_best_only=True,
                           save_weights_only=True,
                           mode='auto',
                           period=1),
           ReduceLROnPlateau(monitor='val_loss',
                             factor=0.5,
                             patience=0,
                             epsilon=0.001,
                             cooldown=0)
          ],
      validation_data = valid_generator,
      max_queue_size=10
)

In [ ]:
def visualize_history(history):
    plt.figure(figsize=(16,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['acc'][1:])
    plt.plot(history.history['val_acc'][1:])
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train','Validation'], loc='upper left')

    plt.title('model accuracy')

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'][1:])
    plt.plot(history.history['val_loss'][1:])
    plt.ylabel('val_loss')
    plt.xlabel('epoch')
    plt.legend(['train','Validation'], loc='upper left')
    plt.title('model loss')
    gc.collect()

In [ ]:
visualize_history(history)

In [ ]:
def plot_results(index):
    
    X, y_true = valid_generator.__getitem__(index)
        
    fig, ax = plt.subplots(8, 2, figsize = (40, 60))

    y_pred = model.predict(X)
    count = 0

    for i in range(len(X)):
        img = X[i]
        label_true = y_true[i]
        label_pred = y_pred[i]

        ax[i][count].imshow(img)
        true_idxs = np.where(label_true == 1)[0]
        true_label = [classes_names[int(classes[j])] for j in true_idxs]
        ax[i][count].set_title(true_label)
        ax[i][count].axis('off')

        idxs = np.argsort(label_pred)[::-1][:5]
        labels = []

        for (k, j) in enumerate(idxs):
            label = "{}: {:.2f}%".format(classes_names[int(classes[j])], label_pred[j] * 100)
            labels.append(label)

        ax[i][count + 1].imshow(img)
        ax[i][count + 1].set_title(labels)
        ax[i][count + 1].axis('off')

    plt.show()

In [ ]:
plot_results(0)